In [1]:
import findspark
findspark.init()

from pyspark.sql import SparkSession;

spark = SparkSession.builder.master("local[4]").appName("ISM6562 Spark App01").enableHiveSupport().getOrCreate();

# Let's get the SparkContext object. It's the entry point to the Spark API. It's created when you create a sparksession
sc = spark.sparkContext  

# note: If you have multiple spark sessions running (like from a previous notebook you've run), 
# this spark session webUI will be on a different port than the default (4040). One way to 
# identify this part is with the following line. If there was only one spark session running, 
# this will be 4040. If it's higher, it means there are still other spark sesssions still running.
spark_session_port = spark.sparkContext.uiWebUrl.split(":")[-1]
print("Spark Session WebUI Port: " + spark_session_port)

23/04/09 19:39:58 WARN Utils: Your hostname, MBP-SMITH515.local resolves to a loopback address: 127.0.0.1; using 192.168.4.81 instead (on interface en0)
23/04/09 19:39:58 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


23/04/09 19:39:58 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
23/04/09 19:39:59 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
Spark Session WebUI Port: 4041


In [2]:
# this will set the log level to ERROR. This will hide the INFO or WARNING messages that are printed out by default. If you want to see them, set this to INFO or WARN.
sc.setLogLevel("ERROR")

In [3]:
spark

In [4]:
def parseLine(line):
    fields = line.split(',')
    stationID = fields[0]
    entryType = fields[2]
    temperature = float(fields[3]) * 0.1 * (9.0 / 5.0) + 32.0
    return (stationID, entryType, temperature)

In [5]:
lines = sc.textFile("data/1800.csv")
parsedLines = lines.map(parseLine) # parsedLines = (stationID, entryType, temperature)
maxTemps = parsedLines.filter(lambda x: "TMAX" in x[1])  # maxTemps = (stationID, "TMAX", temperature)
stationTemps = maxTemps.map(lambda x: (x[0], x[2]))  # stationTemps = (stationID, temperature)
maxTemps = stationTemps.reduceByKey(lambda x, y: max(x,y)) # maxTemps = (stationID, maxTemperature)

In [6]:
results = maxTemps.collect();
 
for result in results:
    print(f"{result[0]}\t{result[1]:.2f}")

ITE00100554	90.14
EZE00100082	90.14
